In [1]:
import scipy.io
import numpy as np
import networkx as nx

In [10]:
fname1 = 'fungal_networks_data/Conductance_Text_Files/Sc_M_I_U_N_42d_1.txt'

# read in the network file (an edge list)
print(f'reading the network file : {fname1}\n')
G = nx.read_weighted_edgelist(fname1, nodetype=str)

n = len(G.nodes())
m = len(G.edges())

print("Number of nodes (n):", n)
print("Number of edges (m):", m)

reading the network file : fungal_networks_data/Conductance_Text_Files/Sc_M_I_U_N_42d_1.txt

Number of nodes (n): 536
Number of edges (m): 689
